In [1]:
import pandas as pd
import numpy as np

In [4]:
df = pd.read_excel('/content/Disponibilidad_Real.xlsx')
df.head()

,Fecha,Recurso,Tipo Generación,Código Agente,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,Archivo,Código Recurso
0,2000-01-01,BAJO ANCHICAYA,HIDRAULICA,EPSG,13000.0,13000.0,13000.0,13000.0,13000.0,13000.0,13000.0,13000.0,13000.0,13000.0,13000.0,13000.0,13000.0,13000.0,13000.0,13000.0,13000.0,13000.0,13000.0,13000.0,13000.0,13000.0,13000.0,13000.0,Disponibilidad Real (kW) 2000,NaN
1,2000-01-01,BETANIA,HIDRAULICA,CHBG,540000.0,540000.0,540000.0,540000.0,540000.0,540000.0,540000.0,540000.0,540000.0,540000.0,540000.0,540000.0,540000.0,540000.0,540000.0,540000.0,540000.0,540000.0,540000.0,540000.0,540000.0,540000.0,540000.0,540000.0,Disponibilidad Real (kW) 2000,NaN
2,2000-01-01,CALIMA,HIDRAULICA,EPSG,120000.0,120000.0,120000.0,120000.0,120000.0,120000.0,120000.0,120000.0,120000.0,120000.0,120000.0,120000.0,120000.0,120000.0,120000.0,120000.0,120000.0,120000.0,120000.0,120000.0,120000.0,120000.0,120000.0,120000.0,Disponibilidad Real (kW) 2000,NaN
3,2000-01-01,CASALCO,HIDRAULICA,EMGG,544000.0,544000.0,544000.0,544000.0,544000.0,544000.0,544000.0,544000.0,544000.0,544000.0,544000.0,544000.0,544000.0,544000.0,544000.0,544000.0,544000.0,544000.0,544020.0,544000.0,544000.0,544010.0,544000.0,544000.0,Disponibilidad Real (kW) 2000,NaN
4,2000-01-01,CHIVOR,HIDRAULICA,CHVG,750000.0,750000.0,750000.0,750000.0,750000.0,750000.0,750000.0,750000.0,750000.0,750000.0,750000.0,750000.0,750000.0,750000.0,750000.0,750000.0,750000.0,750000.0,750000.0,750000.0,750000.0,750000.0,750000.0,750000.0,Disponibilidad Real (kW) 2000,NaN


In [5]:
df['Tipo Generación'].value_counts()

TERMICA       274307
HIDRAULICA    176887
SOLAR            731
Name: Tipo Generación, dtype: int64

In [8]:
list_of_generator_type = df['Tipo Generación'].unique()
list_of_generator_type

array(['HIDRAULICA', 'TERMICA', 'SOLAR'], dtype=object)

In [11]:

def melt_df(df):
    #Hours
#     hours= [i for i in range(0,24)]
    hours = list(range(0, 24))
    
    #Convert hours from integers to string, column names
    hours_str = list(map(str, hours))
            
    #Melt original dataframe
    df_melt = pd.melt(df,
                      id_vars=['Fecha', 'Recurso', 'Código Recurso', 'Tipo Generación','Código Agente', 'Archivo'],
                      value_vars=hours_str,
                      var_name='Hour',
                      value_name='Disponibilidad')
    
    #Convert string to float
    df_melt["Hour"] = pd.to_numeric(df_melt["Hour"])
    
    #Add hours to the date which only had considered the day
    df_melt['Fecha'] = pd.to_datetime(df_melt['Fecha'], format="%Y-%m-%d")
    df_melt["Fecha"] = df_melt["Fecha"] + pd.to_timedelta(df_melt["Hour"],unit='H')
    
    #Order de data
    df_melt.sort_values(by=['Fecha'], inplace=True, ascending=True)
    
    #Reset index
    df_melt=df_melt.reset_index(drop=True).drop('Hour', axis = 1).set_index('Fecha')
    
    df_melt['Date'] = df_melt.index.date
    df_melt['Year'] = df_melt.index.year
    df_melt['Month'] = df_melt.index.month
    df_melt['Day'] = df_melt.index.day
    df_melt['Weekday'] = df_melt.index.weekday # Monday=0
    df_melt['Hour'] = df_melt.index.hour

    return df_melt

df = melt_df(df)
df

,Recurso,Código Recurso,Tipo Generación,Código Agente,Archivo,Disponibilidad,Date,Year,Month,Day,Weekday,Hour
Fecha,,,,,,,,,,,,
2000-01-01 00:00:00,BAJO ANCHICAYA,NaN,HIDRAULICA,EPSG,Disponibilidad Real (kW) 2000,13000.0,2000-01-01,2000,1,1,5,0
2000-01-01 00:00:00,ZIPAEMG 5,NaN,TERMICA,EMGG,Disponibilidad Real (kW) 2000,62000.0,2000-01-01,2000,1,1,5,0
2000-01-01 00:00:00,CARTAGENA 1,NaN,TERMICA,TRCG,Disponibilidad Real (kW) 2000,59000.0,2000-01-01,2000,1,1,5,0
2000-01-01 00:00:00,CARTAGENA 2,NaN,TERMICA,TRCG,Disponibilidad Real (kW) 2000,47000.0,2000-01-01,2000,1,1,5,0
2000-01-01 00:00:00,CARTAGENA 3,NaN,TERMICA,TRCG,Disponibilidad Real (kW) 2000,63000.0,2000-01-01,2000,1,1,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...
2021-07-15 23:00:00,PORCE II,NaN,HIDRAULICA,EPMG,Disponibilidad Real (kW) 2021,0.0,2021-07-15,2021,7,15,3,23
2021-07-15 23:00:00,PLAYAS,NaN,HIDRAULICA,EPMG,Disponibilidad Real (kW) 2021,0.0,2021-07-15,2021,7,15,3,23
2021-07-15 23:00:00,PAGUA,NaN,HIDRAULICA,ENDG,Disponibilidad Real (kW) 2021,0.0,2021-07-15,2021,7,15,3,23


In [12]:
df_TERMICA = df[df['Tipo Generación'] == 'TERMICA']
df_HIDRAULICA = df[df['Tipo Generación'] == 'HIDRAULICA']
df_SOLAR = df[df['Tipo Generación'] == 'SOLAR']

In [13]:
ts_TERMICA = df_TERMICA[['Disponibilidad']].groupby(df_TERMICA.Date).agg('sum')
ts_HIDRAULICA = df_HIDRAULICA[['Disponibilidad']].groupby(df_HIDRAULICA.Date).agg('sum')
ts_SOLAR = df_SOLAR[['Disponibilidad']].groupby(df_SOLAR.Date).agg('sum')

In [19]:
ts_TERMICA.columns = ['Disponibilidad_TERMICA']
ts_HIDRAULICA.columns = ['Disponibilidad_HIDRAULICA']
ts_SOLAR.columns = ['Disponibilidad_SOLAR']

In [22]:
merge1 = ts_TERMICA.merge(ts_HIDRAULICA, how = 'left', on = 'Date')
ts_Availability = merge1.merge(ts_SOLAR, how = 'left', on = 'Date' )
ts_Availability

,Disponibilidad_TERMICA,Disponibilidad_HIDRAULICA,Disponibilidad_SOLAR
Date,,,
2000-01-01,81606180.0,172681050.0,NaN
2000-01-02,79419200.0,172503990.0,NaN
2000-01-03,79410200.0,172386230.0,NaN
2000-01-04,79403200.0,171953670.0,NaN
2000-01-05,79403200.0,172344340.0,NaN
...,...,...,...
2021-07-11,106324499.0,233070749.0,NaN
2021-07-12,106382000.0,233112748.0,NaN
2021-07-13,106282799.0,235541246.0,NaN


In [23]:
ts_Availability.to_csv('ts_Availability.csv')